# Merge OPM Survey Data with CNN Features

## Setup

In [ ]:
# Load packages
library(tidyverse)

# Set up sagemaker
library(reticulate)
sagemaker <- import('sagemaker')
session <- sagemaker$Session()

# s3 bucket name
bucket = "worldbank-pakistan-data"
local_dir = '/home/ec2-user/SageMaker/'

## Load Data

In [ ]:
session$download_data(path = local_dir,
              bucket = bucket,
              key_prefix = 'OPM/FinalData/Individual Datasets/bisp_socioeconomic.Rds')


In [ ]:
opm_df <- readRDS(file.path(local_dir, 'bisp_socioeconomic.Rds'))

cnn_df <- session$read_s3_file(bucket,
                     'OPM/FinalData/Individual Datasets/bisp_cnn_features_all_Nbands3_nNtlBins3_minNTLbinCount16861.csv')

## Merge

In [ ]:
cnn_df <- cnn_df %>%
  dplyr::select(-X)

opm_df <- merge(opm_df, cnn_df, by = c("uid"), all.x=T, all.y=F)

## Export

In [ ]:
write.csv(opm_df, file.path(local_dir, "cnn_merge.csv"), row.names = F)

session$upload_data(path       = file.path(local_dir, "cnn_merge.csv"), 
                    bucket     = bucket, 
                    key_prefix = file.path("OPM", "FinalData", "Merged Datasets"))
